In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from datetime import date
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

from config import NEWS_API_KEY
from api_helpers import *
from sim import *

In [24]:
data = news_top()

In [4]:
queries = []
for article in data['articles']:
    query = article['title']
    if article['description'] is not None:
        query += article['description']
        
    query = query.replace(article['source']['name'], '') # remove publication information
    queries.append(query)
queries

['Melinda Gates could become world’s second-richest woman - Lack of prenuptial agreement with Bill Gates could herald $73bn divorce settlement as fears focus on future of couple’s charity',
 "NFC East draft grades: Eagles jump up ... and Dave Gettleman trades down! - NFL.comIn the wake of an embarrassing season in the NFC East, Dallas, New York, Philadelphia and Washington entered the 2021 NFL Draft with plenty of room for improvement. How does each team's haul grade out? Gennaro Filice has the division's report card.",
 'Dozens of businesses speak out against voting restrictions in Texas - At least 50 businesses including Hewlett-Packard, Patagonia, American Airlines, and P. Terry’s have joined together to speak out against voting restrictions in Texas.',
 'Your COVID vaccine booster might be a patch or a pill. Or it might come with your flu shot. - The next generation of COVID vaccines may come in different forms and might even help prevent future pandemics.',
 'San Francisco moves t

In [5]:
# from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize 
# import nltk
# # nltk.download('punkt')
# # nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# # stop_words = set(stopwords.words('english')) 
  
# tokens = word_tokenize(query) 
  
# # filtered_sentence = [w for w in word_tokens if not w in stop_words]
# tagged = nltk.pos_tag(tokens)
# tagged
# filtered = [w[0] for w in tagged if w[1].startswith('N')]
# filtered

In [6]:
# for ent in doc.ents:
#     print(ent, ent.label_)

In [13]:
table_data = []
for qid, query in enumerate(queries):
    print('------------------')
    print(query)

    # start to break query into named entities
    doc = nlp(query)

    entities = []
    for ent in doc.ents:
        if (ent.label_ in ['PERSON', 'FAC', 'ORG', 'NORP', 'PRODUCT']) and (ent.text not in entities):
            if ent.label_ == 'PERSON':
                label = 'PERSON'
            elif ent.label_ == 'NORP':
                label = 'GROUP'
            elif ent.label_ == 'ORG':
                label = 'ORGANIZATION'
            else:
                label = ''

        articledata = wiki_query(ent.text + label)
        
        try: 
            search_results= articledata['query']['search']
            i = 0
            while((i < len(search_results)) & (distance(ent.text, search_results[i]['title']) < 10)):
                title = search_results[i]['title']
                info = wiki_pageinfo(title)
                info = list(info['query']['pages'].values())[0]
                
                categories = [kv['title'] for kv in info['categories'] if 'births' not in kv['title']]
                
                # ignore diambiguation pages
                if 'Category:Disambiguation pages' in categories:
                    pass
                else:
                    content = list(wiki_pagecontent(title)['query']['pages'].values())[0]
                    if 'thumbnail' in content:
                        image = content['thumbnail']['source']
                    else:
                        image = None
                    table_data.append({'news_id': qid,
                                       'entity': ent.text,
                                       'label': label,
                                       'title': info['title'],
                                       'revised': info['touched'],
                                       'url': info['fullurl'],
                                       'wiki': content['extract'],
                                       'image': image})
                i += 1
        
        except(IndexError):
            continue

------------------
Melinda Gates could become world’s second-richest woman - Lack of prenuptial agreement with Bill Gates could herald $73bn divorce settlement as fears focus on future of couple’s charity
------------------
NFC East draft grades: Eagles jump up ... and Dave Gettleman trades down! - NFL.comIn the wake of an embarrassing season in the NFC East, Dallas, New York, Philadelphia and Washington entered the 2021 NFL Draft with plenty of room for improvement. How does each team's haul grade out? Gennaro Filice has the division's report card.
------------------
Dozens of businesses speak out against voting restrictions in Texas - At least 50 businesses including Hewlett-Packard, Patagonia, American Airlines, and P. Terry’s have joined together to speak out against voting restrictions in Texas.
------------------
Your COVID vaccine booster might be a patch or a pill. Or it might come with your flu shot. - The next generation of COVID vaccines may come in different forms and might

------------------
Can California reach herd immunity before the rest of the country? - SF GateCalifornia is in a good spot with a population that mostly wants the vaccine.
------------------
Apple hires former Google AI scientist who left after ethics turmoil - Apple has hired former Google Brain research manager Samy Bengio, who left the company after its firings of two female AI researchers..
------------------
After Israel stampede, some ultra-Orthodox are looking at their role in the tragedy - Days after a deadly stampede resulted in the deaths of 45 people at a religious festival in northern Israel, many are now asking who is at fault.
------------------
Will Smith comes clean: 'I'm in the worst shape of my life' - Will Smith flaunts his pandemic-induced Dad Bod, earning snark from Questlove and Michael Bay. At least Ava DuVernay and Arnold Schwarzenegger were nice.


In [17]:
table = pd.DataFrame(table_data)
table

,news,entity,label,title,revised,url,wiki,image
0,0,Melinda Gates,ORGANIZATION,Melinda Gates,2021-05-04T19:43:17Z,https://en.wikipedia.org/wiki/Melinda_Gates,Melinda Ann Gates (née French; born August 15...,https://upload.wikimedia.org/wikipedia/commons...
1,0,Bill Gates,PERSON,Bill Gates,2021-05-04T19:42:25Z,https://en.wikipedia.org/wiki/Bill_Gates,"William Henry Gates III (born October 28, 195...",https://upload.wikimedia.org/wikipedia/commons...
2,0,73bn,PERSON,Ireland,2021-05-04T21:09:30Z,https://en.wikipedia.org/wiki/Ireland,Ireland ( (listen); Irish: Éire [ˈeːɾʲə] (list...,https://upload.wikimedia.org/wikipedia/commons...
3,0,73bn,PERSON,Nuremberg,2021-05-04T21:09:09Z,https://en.wikipedia.org/wiki/Nuremberg,Nuremberg ( NEWR-əm-burg; German: Nürnberg [ˈn...,https://upload.wikimedia.org/wikipedia/commons...
4,1,Dave Gettleman,PERSON,Gettleman,2021-04-21T19:19:37Z,https://en.wikipedia.org/wiki/Gettleman,Gettleman is a surname.,None
...,...,...,...,...,...,...,...,...
153,35,Google Brain,ORGANIZATION,Google,2021-05-02T06:59:34Z,https://en.wikipedia.org/wiki/Google,Google LLC is an American multinational techno...,https://upload.wikimedia.org/wikipedia/commons...
154,36,Israel,ORGANIZATION,Israel,2021-05-02T22:07:47Z,https://en.wikipedia.org/wiki/Israel,Israel (; Hebrew: יִשְׂרָאֵל‎; Arabic: إِسْرَا...,https://upload.wikimedia.org/wikipedia/commons...
155,37,Will Smith,PERSON,Will Smith,2021-05-04T01:53:45Z,https://en.wikipedia.org/wiki/Will_Smith,"Willard Carroll Smith Jr. (born September 25, ...",https://upload.wikimedia.org/wikipedia/commons...
156,37,Ava DuVernay,PERSON,Ava DuVernay,2021-05-04T01:53:39Z,https://en.wikipedia.org/wiki/Ava_DuVernay,"Ava Marie DuVernay (; born August 24, 1972) is...",https://upload.wikimedia.org/wikipedia/commons...


In [20]:
today = date.today().strftime("%b-%d-%Y")
table.to_csv(f'{today}-us-top-headlines.csv', index = False)

In [28]:
news_table = pd.DataFrame(queries).reset_index()
news_table.columns = ['news_id', 'news']